## Data cleaning as detoxify input

In [1]:
import pandas as pd

In [2]:
import datetime as dt

### After comments

In [3]:
fds_comments_after = pd.read_csv('./data/fds_comments_after.csv')

C:\Users\nguye\AppData\Local\Temp\ipykernel_14960\2756075641.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  fds_comments_after = pd.read_csv('fds_comments_after.csv')


In [4]:
fds_comments_after.head()

,all_awardings,associated_award,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,...,steward_reports,stickied,subreddit,subreddit_id,total_awards_received,distinguished,collapsed_because_crowd_control,edited,author_premium,author_cakeday
0,[],NaN,extraacct1234,#46d160,NaN,[],986e573c-f8d6-11e9-b94f-0e1dbdeefc72,FDS Newbie,dark,text,...,[],False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN
1,[],NaN,smarfette,#46d160,NaN,[],986e573c-f8d6-11e9-b94f-0e1dbdeefc72,FDS Newbie,dark,text,...,[],False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN
2,[],NaN,[deleted],NaN,NaN,NaN,NaN,NaN,dark,NaN,...,[],False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN
3,[],NaN,AutoModerator,NaN,NaN,[],NaN,NaN,NaN,text,...,[],True,FemaleDatingStrategy,t5_xaiot,0,moderator,NaN,NaN,NaN,NaN
4,[],NaN,_s0ul99,#46d160,NaN,[],986e573c-f8d6-11e9-b94f-0e1dbdeefc72,FDS Newbie,dark,text,...,[],False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN


In [5]:
#See all columns names
list(fds_comments_after)

['all_awardings',
 'associated_award',
 'author',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_template_id',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'awarders',
 'body',
 'created_utc',
 'gildings',
 'id',
 'is_submitter',
 'link_id',
 'locked',
 'no_follow',
 'parent_id',
 'permalink',
 'retrieved_on',
 'score',
 'send_replies',
 'steward_reports',
 'stickied',
 'subreddit',
 'subreddit_id',
 'total_awards_received',
 'distinguished',
 'collapsed_because_crowd_control',
 'edited',
 'author_premium',
 'author_cakeday']

In [6]:
#Only choose what relevant
fds_comments_after_df = fds_comments_after[['body','author','created_utc','retrieved_on','permalink','parent_id','subreddit','score']]

In [7]:
#Change epoch time to human time 
fds_comments_after_df['created_utc'] = pd.to_datetime(fds_comments_after_df['created_utc'], unit='s')
fds_comments_after_df['retrieved_on'] = pd.to_datetime(fds_comments_after_df['retrieved_on'], unit='s')

C:\Users\nguye\AppData\Local\Temp\ipykernel_14960\2720722081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_comments_after_df['created_utc'] = pd.to_datetime(fds_comments_after_df['created_utc'], unit='s')
C:\Users\nguye\AppData\Local\Temp\ipykernel_14960\2720722081.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_comments_after_df['retrieved_on'] = pd.to_datetime(fds_comments_after_df['retrieved_on'], unit='s')


In [9]:
#Get month (Oct and Nov)
# Two conditions, to check if data is what I want
fds_comments_after_df[(fds_comments_after_df['created_utc'].dt.month == 10) & (fds_comments_after_df['created_utc'].dt.day == 28)]

,body,author,created_utc,retrieved_on,permalink,parent_id,subreddit,score
14995,[removed],[deleted],2019-10-28 23:33:05,2019-10-29 01:27:28,/r/FemaleDatingStrategy/comments/dm5epg/heres_...,t3_dm5epg,FemaleDatingStrategy,1
14996,[removed],[deleted],2019-10-28 23:31:00,2019-10-29 01:25:54,/r/FemaleDatingStrategy/comments/do8wn7/your_w...,t1_f5kwoh5,FemaleDatingStrategy,1
14997,Reminder that this sub is **FEMALE ONLY**. All...,AutoModerator,2019-10-28 23:30:24,2019-10-29 01:25:28,/r/FemaleDatingStrategy/comments/dohevr/just_r...,t3_dohevr,FemaleDatingStrategy,1
14998,Reminder that this sub is **FEMALE ONLY**. All...,AutoModerator,2019-10-28 23:28:58,2019-10-29 01:24:20,/r/FemaleDatingStrategy/comments/dohe7x/shout_...,t3_dohe7x,FemaleDatingStrategy,1
14999,"This is my biggest struggle right now, but it'...",LittleBoBooBoo,2019-10-28 23:28:49,2019-10-29 01:24:12,/r/FemaleDatingStrategy/comments/do375c/the_im...,t3_do375c,FemaleDatingStrategy,1
...,...,...,...,...,...,...,...,...
100934,Most men want to waste your time and use you f...,throwawayy92838383,2019-10-28 16:44:54,2019-10-28 16:49:43,/r/FemaleDatingStrategy/comments/doa03p/a_remi...,t1_f5lkx90,FemaleDatingStrategy,1
100935,Yep it’s the true 50/50✨,dispositiondesxyz,2019-10-28 16:42:44,2019-10-28 16:47:22,/r/FemaleDatingStrategy/comments/dnwd12/look_a...,t1_f5jk3mo,FemaleDatingStrategy,1
100936,Welcome!,TheOGJammies,2019-10-28 16:40:57,2019-10-28 16:45:34,/r/FemaleDatingStrategy/comments/dob9vz/i_just...,t3_dob9vz,FemaleDatingStrategy,1
100937,"Oh wow, tunics look so cute! Are there any sh...",Meredeen,2019-10-28 16:40:41,2019-10-28 16:45:19,/r/FemaleDatingStrategy/comments/do2s08/doc_ma...,t1_f5lmivz,FemaleDatingStrategy,1


In [10]:
#Get month (Oct and Nov) - 1 mo after bot; Dec - 2 mo after bot; Jan - 3 mo after bot
fds_comments_after_1mo_df = fds_comments_after_df[(fds_comments_after_df['created_utc'].dt.month == 10) | (fds_comments_after_df['created_utc'].dt.month == 11)]
fds_comments_after_2mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 12]
fds_comments_after_3mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 1]

In [12]:
#Save datasets
fds_comments_after_1mo_df.to_csv('fds_comments_after_1mo.csv')
fds_comments_after_2mo_df.to_csv('fds_comments_after_2mo.csv')
fds_comments_after_3mo_df.to_csv('fds_comments_after_3mo.csv')

### Before comments
Replicate the steps above

In [24]:
fds_comments_before = pd.read_csv('fds_comments_before.csv')

,all_awardings,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_template_id,author_flair_text,author_flair_text_color,author_flair_type,author_fullname,...,stickied,subreddit,subreddit_id,total_awards_received,author_cakeday,edited,steward_reports,distinguished,awarders,associated_award
0,[],CassidyMain,NaN,NaN,[],NaN,NaN,NaN,text,t2_3nrlnhus,...,False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN,NaN
1,[],CassidyMain,NaN,NaN,[],NaN,NaN,NaN,text,t2_3nrlnhus,...,False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN,NaN
2,[],CassidyMain,NaN,NaN,[],NaN,NaN,NaN,text,t2_3nrlnhus,...,False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN,NaN
3,[],gimmethejugs3434,NaN,NaN,[],NaN,NaN,NaN,text,t2_3odknxtj,...,False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN,NaN
4,[],gimmethejugs3434,NaN,NaN,[],NaN,NaN,NaN,text,t2_3odknxtj,...,False,FemaleDatingStrategy,t5_xaiot,0,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
fds_comments_before_df = fds_comments_before[['body','author','created_utc','retrieved_on','permalink','parent_id','subreddit','score']]

In [28]:
#Change epoch time to human time 
fds_comments_before_df['created_utc'] = pd.to_datetime(fds_comments_before_df['created_utc'], unit='s')
fds_comments_before_df['retrieved_on'] = pd.to_datetime(fds_comments_before_df['retrieved_on'], unit='s')

C:\Users\nguye\AppData\Local\Temp\ipykernel_14960\2965675394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_comments_before_df['created_utc'] = pd.to_datetime(fds_comments_before_df['created_utc'], unit='s')
C:\Users\nguye\AppData\Local\Temp\ipykernel_14960\2965675394.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_comments_before_df['retrieved_on'] = pd.to_datetime(fds_comments_before_df['retrieved_on'], unit='s')


In [32]:
#Check if the date range is what I want
fds_comments_before_df[(fds_comments_before_df['created_utc'].dt.month == 10) & (fds_comments_before_df['created_utc'].dt.day == 27)]

,body,author,created_utc,retrieved_on,permalink,parent_id,subreddit,score
900,Reminder that this sub is **FEMALE ONLY**. All...,AutoModerator,2019-10-27 11:59:01,2019-10-27 11:59:13,/r/FemaleDatingStrategy/comments/dnsay0/id_nev...,t3_dnsay0,FemaleDatingStrategy,1
901,Save it for someone who gives a shit. Enjoy th...,_HEDONISM_BOT,2019-10-27 11:58:02,2019-10-27 11:58:13,/r/FemaleDatingStrategy/comments/dnrkad/daily_...,t1_f5f04sg,FemaleDatingStrategy,1
902,Did he really come of as a douche though?,L8zin,2019-10-27 11:57:39,2019-10-27 11:57:50,/r/FemaleDatingStrategy/comments/d9xkqr/this_s...,t1_f49g5ax,FemaleDatingStrategy,1
903,"We are all ""nice girls"" on this sub, y u b fuc...",Genghis-Gas,2019-10-27 11:57:27,2019-10-27 11:57:38,/r/FemaleDatingStrategy/comments/dns9ar/ladies...,t3_dns9ar,FemaleDatingStrategy,1
904,Right? This sub is horrifying.,Derpizzle,2019-10-27 11:56:47,2019-10-27 11:57:12,/r/FemaleDatingStrategy/comments/dju7ep/time_t...,t1_f5emwmi,FemaleDatingStrategy,1
...,...,...,...,...,...,...,...,...
22612,[removed],[deleted],2019-10-27 04:19:42,2019-10-27 04:19:53,/r/FemaleDatingStrategy/comments/d2lg9t/girl_g...,t1_f0i7vw5,FemaleDatingStrategy,1
22613,Thanks for the quick response! Will do :),Jenn_There_Done_That,2019-10-27 04:18:56,2019-10-27 04:19:07,/r/FemaleDatingStrategy/comments/dngfc2/amen/f...,t1_f5d4dv2,FemaleDatingStrategy,1
22614,Damn right! Know your worth!!,sweatydeath,2019-10-27 04:18:51,2019-10-27 04:19:02,/r/FemaleDatingStrategy/comments/dnnboy/former...,t1_f5cxky6,FemaleDatingStrategy,1
22615,Believe me some guys actually try to use post-...,sweatydeath,2019-10-27 04:18:17,2019-10-27 04:18:29,/r/FemaleDatingStrategy/comments/dnkvm9/ditchi...,t1_f5cyiry,FemaleDatingStrategy,1


In [33]:
#Get month
fds_comments_before_1mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 10]
fds_comments_before_2mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 9]
fds_comments_before_3mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 8]
fds_comments_before_4mo_df = fds_comments_after_df[fds_comments_after_df['created_utc'].dt.month == 7]

In [34]:
#Save datasets
fds_comments_before_1mo_df.to_csv('fds_comments_before_1mo.csv')
fds_comments_before_2mo_df.to_csv('fds_comments_before_2mo.csv')
fds_comments_before_3mo_df.to_csv('fds_comments_before_3mo.csv')
fds_comments_before_4mo_df.to_csv('fds_comments_before_4mo.csv')

## Submission Before

In [5]:
fds_sub_b4 = pd.read_csv('./data/fds_submissions_before.csv')

C:\Windows\Temp\ipykernel_12804\262336955.py:1: DtypeWarning: Columns (33,34,45,59,60,87,88,89) have mixed types. Specify dtype option on import or set low_memory=False.
  fds_sub_b4 = pd.read_csv('./data/fds_submissions_before.csv')


In [6]:
fds_sub_b4.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,awarders,...,edited,media_metadata,treatment_tags,upvote_ratio,url_overridden_by_dest,is_gallery,gallery_data,distinguished,removed_by,poll_data
0,[],False,RoughPotential,NaN,[],NaN,text,t2_2dwt4rru,False,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,PhilSwift141,NaN,[],NaN,text,t2_4lz88y5k,False,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,TheOGJammies,NaN,[],NaN,text,t2_43ex46gt,False,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,TheOGJammies,NaN,[],NaN,text,t2_43ex46gt,False,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[],False,CSQUITO,NaN,[],FDS Disciple,text,t2_wrk3i,False,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#See all columns names
list(fds_sub_b4)

['all_awardings',
 'allow_live_comments',
 'author',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_type',
 'author_fullname',
 'author_patreon_flair',
 'awarders',
 'can_mod_post',
 'contest_mode',
 'created_utc',
 'domain',
 'full_link',
 'gildings',
 'id',
 'is_crosspostable',
 'is_meta',
 'is_original_content',
 'is_reddit_media_domain',
 'is_robot_indexable',
 'is_self',
 'is_video',
 'link_flair_background_color',
 'link_flair_richtext',
 'link_flair_text_color',
 'link_flair_type',
 'locked',
 'media_only',
 'no_follow',
 'num_comments',
 'num_crossposts',
 'og_description',
 'og_title',
 'over_18',
 'parent_whitelist_status',
 'permalink',
 'pinned',
 'pwls',
 'retrieved_on',
 'score',
 'selftext',
 'send_replies',
 'spoiler',
 'steward_reports',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_subscribers',
 'subreddit_type',
 'suggested_sort',
 'thumbnail',
 'title',
 'total_awards_received',
 'updated_utc',
 'url',
 'whitelis

In [13]:
fds_sub_b4_df = fds_sub_b4[['author','author_flair_text','created_utc','retrieved_on','domain','full_link','id','is_reddit_media_domain','permalink','is_video','locked','num_comments','subreddit','subreddit_id','score','selftext','subreddit_subscribers','title','total_awards_received','updated_utc','removed_by','poll_data']]

In [14]:
fds_sub_b4_df.head()

,author,author_flair_text,created_utc,retrieved_on,domain,full_link,id,is_reddit_media_domain,permalink,is_video,...,subreddit,subreddit_id,score,selftext,subreddit_subscribers,title,total_awards_received,updated_utc,removed_by,poll_data
0,RoughPotential,NaN,1572142154,1572142155,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,dnnboy,False,/r/FemaleDatingStrategy/comments/dnnboy/former...,False,...,FemaleDatingStrategy,t5_xaiot,0,I have a guy who failed all of my tests and go...,25351,Former Lover Turned “Friend”,0,1.572298e+09,NaN,NaN
1,PhilSwift141,NaN,1572140831,1572140833,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,dnn34z,False,/r/FemaleDatingStrategy/comments/dnn34z/m_a_n/,False,...,FemaleDatingStrategy,t5_xaiot,1,What if...man\nHumor,25351,m a n,0,1.572298e+09,NaN,NaN
2,TheOGJammies,NaN,1572139922,1572139923,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,dnmx78,False,/r/FemaleDatingStrategy/comments/dnmx78/why_no...,False,...,FemaleDatingStrategy,t5_xaiot,0,MGTOWs love to project how they’re somehow pun...,25351,Why No Woman Should Fear “Dying Alone”.,0,1.572298e+09,NaN,NaN
3,TheOGJammies,NaN,1572138485,1572138522,self.datingoverforty,https://www.reddit.com/r/FemaleDatingStrategy/...,dnmns4,False,/r/FemaleDatingStrategy/comments/dnmns4/anothe...,False,...,FemaleDatingStrategy,t5_xaiot,0,NaN,25348,Another Reminder To Stop Trying To Save Men Fr...,0,1.572297e+09,NaN,NaN
4,CSQUITO,FDS Disciple,1572138411,1572138412,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,dnmn96,False,/r/FemaleDatingStrategy/comments/dnmn96/you_ca...,False,...,FemaleDatingStrategy,t5_xaiot,0,Friend \nFriend w benefits/bootycall\nGirlfrie...,25344,You can only fall into 3 categories,0,1.572297e+09,NaN,NaN


In [15]:
#Change to human date
fds_sub_b4_df['created_utc'] = pd.to_datetime(fds_sub_b4_df['created_utc'], unit='s')
fds_sub_b4_df['retrieved_on'] = pd.to_datetime(fds_sub_b4_df['retrieved_on'], unit='s')
fds_sub_b4_df['updated_utc'] = pd.to_datetime(fds_sub_b4_df['updated_utc'], unit='s')

C:\Windows\Temp\ipykernel_12804\2510439944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_sub_b4_df['created_utc'] = pd.to_datetime(fds_sub_b4_df['created_utc'], unit='s')
C:\Windows\Temp\ipykernel_12804\2510439944.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fds_sub_b4_df['retrieved_on'] = pd.to_datetime(fds_sub_b4_df['retrieved_on'], unit='s')
C:\Windows\Temp\ipykernel_12804\2510439944.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [18]:
#Check if the date range is what I want
fds_sub_b4_df[(fds_sub_b4_df['created_utc'].dt.month == 7) & (fds_sub_b4_df['created_utc'].dt.day == 27)].head()

,author,author_flair_text,created_utc,retrieved_on,domain,full_link,id,is_reddit_media_domain,permalink,is_video,...,subreddit,subreddit_id,score,selftext,subreddit_subscribers,title,total_awards_received,updated_utc,removed_by,poll_data
699,thatlonghairedbitch,NaN,2020-07-27 21:20:35,2020-07-27 21:20:45,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,hz15is,False,/r/FemaleDatingStrategy/comments/hz15is/had_my...,False,...,FemaleDatingStrategy,t5_xaiot,1,[removed],86190,Had my first threesome,0,NaT,NaN,NaN
700,greatflo,FDS Newbie,2020-07-27 21:06:18,2020-07-27 21:06:30,i.redd.it,https://www.reddit.com/r/FemaleDatingStrategy/...,hz0vt5,True,/r/FemaleDatingStrategy/comments/hz0vt5/dumped...,False,...,FemaleDatingStrategy,t5_xaiot,1,NaN,86185,"Dumped a LVM for the first time, cut off all m...",0,NaT,NaN,NaN
701,Cucharamama,FDS Newbie,2020-07-27 20:54:55,2020-07-27 20:55:06,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,hz0o0g,False,/r/FemaleDatingStrategy/comments/hz0o0g/is_it_...,False,...,FemaleDatingStrategy,t5_xaiot,1,[removed],86182,Is it a red flag when guys on OLD ask to hango...,0,NaT,NaN,NaN
702,bonenecklace,FDS Newbie,2020-07-27 20:50:52,2020-07-27 20:51:04,i.redd.it,https://www.reddit.com/r/FemaleDatingStrategy/...,hz0lbc,True,/r/FemaleDatingStrategy/comments/hz0lbc/zero_t...,False,...,FemaleDatingStrategy,t5_xaiot,1,NaN,86182,Zero to one hundred first thing in the morning...,0,NaT,NaN,NaN
703,dior-not-war,FDS Newbie,2020-07-27 20:37:51,2020-07-27 20:38:02,i.redd.it,https://www.reddit.com/r/FemaleDatingStrategy/...,hz0cn4,True,/r/FemaleDatingStrategy/comments/hz0cn4/always...,False,...,FemaleDatingStrategy,t5_xaiot,1,NaN,86178,Always block them so they don’t even get the c...,0,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29732,StressedMacaroon25,FDS Newbie,2020-07-27 21:48:59,2020-07-27 21:49:10,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,hz1okt,False,/r/FemaleDatingStrategy/comments/hz1okt/ladies...,False,...,FemaleDatingStrategy,t5_xaiot,1,[removed],86200,"Ladies, please help me to understand if he's h...",0,NaT,NaN,NaN
29733,kindthoughtsandwords,FDS Newbie,2020-07-27 21:48:53,2020-07-27 21:49:04,self.FemaleDatingStrategy,https://www.reddit.com/r/FemaleDatingStrategy/...,hz1oia,False,/r/FemaleDatingStrategy/comments/hz1oia/im_not...,False,...,FemaleDatingStrategy,t5_xaiot,1,[removed],86200,"""I'm not judging you, I'm just looking at you!""",0,NaT,NaN,NaN
29734,Myplummms,FDS Disciple,2020-07-27 21:47:27,2020-07-27 21:47:38,i.redd.it,https://www.reddit.com/r/FemaleDatingStrategy/...,hz1nmd,True,/r/FemaleDatingStrategy/comments/hz1nmd/youre_...,False,...,FemaleDatingStrategy,t5_xaiot,1,NaN,86200,You're all so lovely! I hope you ladies find s...,0,NaT,NaN,NaN
29735,Myplummms,FDS Disciple,2020-07-27 21:44:03,2020-07-27 21:44:13,i.redd.it,https://www.reddit.com/r/FemaleDatingStrategy/...,hz1lcw,True,/r/FemaleDatingStrategy/comments/hz1lcw/the_re...,False,...,FemaleDatingStrategy,t5_xaiot,1,NaN,86199,The reason these are statements women need to ...,0,NaT,NaN,NaN


In [19]:
#Get month
fds_subm_before_1mo_df = fds_sub_b4_df[fds_sub_b4_df['created_utc'].dt.month == 10]
fds_subm_before_2mo_df = fds_sub_b4_df[fds_sub_b4_df['created_utc'].dt.month == 9]
fds_subm_before_3mo_df = fds_sub_b4_df[fds_sub_b4_df['created_utc'].dt.month == 8]
fds_subm_before_4mo_df = fds_sub_b4_df[fds_sub_b4_df['created_utc'].dt.month == 7]

In [20]:
#Save datasets
fds_subm_before_1mo_df.to_csv('fds_comments_before_1mo.csv')
fds_subm_before_2mo_df.to_csv('fds_comments_before_2mo.csv')
fds_subm_before_3mo_df.to_csv('fds_comments_before_3mo.csv')
fds_subm_before_4mo_df.to_csv('fds_comments_before_4mo.csv')